In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from scipy import stats
from sklearn import linear_model
from sklearn.metrics import mean_squared_error
from sklearn import metrics
import scipy
from sklearn import preprocessing

In [2]:
fmli = pd.read_csv("fmli_15_clean.csv")
fmli.drop(["DETTACHED", "PSU"], axis = 1, inplace = True)

In [3]:
sf2010 = pd.read_csv("fmli_2010_clean.csv")

In [4]:
dm = sf2010[["OWN", "DEGREE", "HHSIZE", "INCOME", "ROOMS", "VEHICLES"]]
dm.replace(".", np.nan, inplace = True)
dm["ROOMS"] = pd.to_numeric(dm["ROOMS"])

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/pandas/core/frame.py:4042: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  method=method)
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until


##### Normalize data frame

In [5]:
# Log every variables, save it in a new data frame for future use.
log = np.log(fmli)
log["OWN"] = fmli["OWN"]
log["DEGREE"] = fmli["DEGREE"]
log["VEHICLES"] = fmli["VEHICLES"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  
/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: invalid value encountered in log
  


In [6]:
# Log every variables, save it in a new data frame for future use.
log_dm = np.log(dm)
log_dm["OWN"] = dm["OWN"]
log_dm["DEGREE"] = dm["DEGREE"]
log_dm["VEHICLES"] = dm["VEHICLES"]
log_dm = log_dm[log_dm.columns[[2, 3, 4, 0, 1, 5]]]
log_dm["CUID"] = sf2010["CUID"]
log_dm["NEWID"] = sf2010["NEWID"]

/Users/winnie/miniconda2/envs/tf/lib/python3.7/site-packages/ipykernel_launcher.py:2: RuntimeWarning: divide by zero encountered in log
  


KeyError: 'CUID'

##### Run models and Apply

In [ ]:
# Apply the same models we got at "Scond model" to sf2010
pred_y = pd.DataFrame()

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.03).fit(X1, Y)
    log_dm.replace(np.Inf, np.nan, inplace = True)
    log_dm.replace(-np.Inf, np.nan, inplace = True)
    log_dm.dropna(inplace = True)
    Xp = log_dm[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    log_dm1 = sm.add_constant(Xp)
    pred_y[log.columns[i]] = np.array(model.predict(log_dm1))

In [ ]:
result = pred_y
for i in pred_y.columns:
    result[i] = np.exp(pred_y[i])

In [ ]:
result["CUID"] = log_dm["CUID"]
result["NEWID"] = log_dm["NEWID"]

In [ ]:
cols = list(result.columns)
cols = [cols[len(cols) - 2]] + [cols[len(cols) - 1]] + cols[:len(cols) - 2]
result = result[cols]

In [ ]:
result.head()

In [ ]:
result.to_csv("pred_fmli_2010.csv")

##### Same procedure for fmli2000

In [ ]:
sf2000 = pd.read_csv("fmli_2000_clean.csv")

In [ ]:
dm = sf2000[["OWN", "DEGREE", "HHSIZE", "INCOME", "ROOMS", "VEHICLES"]]
dm.replace(".", np.nan, inplace = True)
dm["ROOMS"] = pd.to_numeric(dm["ROOMS"])

In [ ]:
# Log every variables, save it in a new data frame for future use.
log_dm = np.log(dm)
log_dm["OWN"] = dm["OWN"]
log_dm["DEGREE"] = dm["DEGREE"]
log_dm["VEHICLES"] = dm["VEHICLES"]
log_dm = log_dm[log_dm.columns[[2, 3, 4, 0, 1, 5]]]
log_dm["NEWID"] = sf2000["NEWID"]

In [ ]:
pred_y = pd.DataFrame()

for i in range(16, len(log.columns)):
    df = log[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES", log.columns[i]]]
    df.replace(np.Inf, np.nan, inplace = True)
    df.replace(-np.Inf, np.nan, inplace = True)
    df.dropna(inplace = True)
    df.reset_index(inplace = True)
    X = df[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    Y = df[log.columns[i]]
    X1 = sm.add_constant(X)
    model = linear_model.Lasso(alpha=0.03).fit(X1, Y)
    log_dm.replace(np.Inf, np.nan, inplace = True)
    log_dm.replace(-np.Inf, np.nan, inplace = True)
    log_dm.dropna(inplace = True)
    Xp = log_dm[["HHSIZE", "INCOME", "ROOMS", "OWN", "DEGREE", "VEHICLES"]]
    log_dm1 = sm.add_constant(Xp)
    pred_y[log.columns[i]] = np.array(model.predict(log_dm1))

In [ ]:
result = pred_y
for i in pred_y.columns:
    result[i] = np.exp(pred_y[i])

In [ ]:
result["NEWID"] = log_dm["NEWID"]

In [ ]:
cols = list(result.columns)
cols = [cols[len(cols) - 1]] + cols[:len(cols) - 1]
result = result[cols]

In [ ]:
result.head()

In [ ]:
result.to_csv("pred_fmli_2000.csv")

##### Expenditures for Each Year

In [ ]:
year4 = pd.read_csv("pred_fmli_sfmsa2015.csv")

In [ ]:
year4.drop(year4.columns[0], axis = 1, inplace = True)